In [ ]:
from tqdm import tqdm

import numpy as np
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors

from astropy.io import fits

from Cthulhu.misc import read_cross_section_file
from Cthulhu.core import summon, compute_cross_section
from Cthulhu.VALD import process_VALD_file

The following four cells are what I used to generate my 1 nbar, 1000 K cross sections for the atomic species of interest in this work. They take quite a while to run and require a bit of set-up to make happen. In particular, the atomic cross sections require that you get a VALD account and request all the relevant datasets first. If you want to modify the cross-sections (e.g. add new atoms or ionization states), you will need to edit these cells and supply the data.

If you are content to use the same cross-sections as I did, then please skip the next four cells and use the provided .npy files instead.

In [ ]:
### GETTING ATOMIC CROSS-SECTIONS FROM THE OLD ONE ###
atoms = ('S','C','O','Na','K')
states = (1,2,3)
state_strs = ('_I','_II','_III')
database = 'VALD'
for atom in tqdm(atoms, desc='Computing atomic cross-sections with Cthulhu...'):
    for state, state_str in zip(states,state_strs):
        try:
            nu_min = 1000                   # 1/cm, equal to 10 micron
            nu_max = 100000                 # 1/cm, equal to 0.1 micron
            process_VALD_file(species = atom, ionization_state = state, VALD_data_dir = './VALD_Line_Lists/')
            summon(database=database, species = atom, VALD_processed_output_dir='./VALD_Line_Lists/', ionization_state = state)

            P = 1e-9                        # Pressure in bars
            T = 1000.0                      # Temperature in Kelvin
            input_directory = './input/'    # Top level directory containing line lists

            compute_cross_section(processed_outputbase=database, species = atom, pressure = P,
                                    temperature = T, input_dir = input_directory,
                                    ionization_state = state, nu_out_min=nu_min, nu_out_max=nu_max,
                                    verbose=True, N_cores=7)
        except:
            # Exception catches anything you don't have data for or isn't supported by cthulhu.
            print("passing on",atom+state_str)
            pass

In [ ]:
### GETTING THE CROSS SECTIONS TO USE FOR THE REST OF THIS NOTEBOOK ###
absorption_dict = {}
full_cross_dict = {}

P = 1e-9
T = 1000


atoms = ('C','O','Na','S','K')
masses = (12.011,15.999,22.990,32.065,39.098) # in amu
database = 'VALD'
for k in tqdm(range(len(atoms)),desc='Getting atomic cross sections...'):
    atom = atoms[k]
    mass = masses[k]
    for state, state_str in zip(states,state_strs):
        # These read out in 1/cm, cm^2
        try:
            nu, sigma = read_cross_section_file(species=atom,database=database,ionization_state=state,
                                                filename=atom+state_str+"_T"+"{:.1F}".format(T)+"K_log_P"+str(round(np.log10(P),1))+"_H2-He_sigma.txt")
        except:
            # Again catching atoms Cthulhu can't support, in this case those were typically K-III and Na-III.
            print("passing on",atom+state_str)
            continue

        # Convert wavenumbers in 1/cm to wavelengths in micron
        waves = np.asarray([10*1000/i for i in nu]) # in um

        full_cross_dict[atom+state_str] = [waves,sigma,mass]

        # Take the strongest line available on the 0.1-10 um range,
        # but bin it down to comparable to resolution elements.
        max_index = np.argmax(sigma)
        if atom+state_str == 'S_I':
            # Hardcoding to pull on the 180.7311 nm line instead, since it is NIST-verified
            max_index = (np.abs(waves - 0.1807311)).argmin()
        max_wave = waves[max_index]

        # For the C, O, S lines, I'm opting for a resolution element of ~0.1 AA.
        # The COS grisms and STIS Echelles can reach dispersions of 0.01 AA per pixel,
        # but I want to bin several resolution elements to increase SNR.
        lower_wave = max_wave - 0.5e-5 # - 0.05 AA
        upper_wave = max_wave + 0.5e-5 # + 0.05 AA

        # For the Na and K lines, I'm going for 2 nm instead.
        # WFC3/UVIS, WFC3/IR, and STIS/IR M/L grisms reach similar or smaller dispersions,
        # but you always want some binning for SNR.
        if atom in ("Na","K"):
            lower_wave = max_wave - 1e-3 # - 1nm
            upper_wave = max_wave + 1e-3 # + 1nm

        upper_index = (np.abs(waves - lower_wave)).argmin()
        lower_index = (np.abs(waves - upper_wave)).argmin()
        print("Getting cross section over {:.0F} indices from wavelengths {:.3F} to {:.3F} AA.".format(np.abs(upper_index - lower_index),
                                                                                                            lower_wave*1e4,
                                                                                                          upper_wave*1e4))
        max_sigma = sigma[lower_index:upper_index]
        max_waves = waves[lower_index:upper_index]
            
        # Define the bandpass in micron
        wmin, wmax = (np.min(max_waves),np.max(max_waves))
        band = wmax - wmin

        # Retrieve the average cross-section on this range
        sec = np.mean(max_sigma)
        wave = np.median(max_waves)
        hw = 0.5*(np.max(max_waves)-np.min(max_waves))

        print("Maximum cross section for {} at index {}, wavenumber {:.0F} cm^-1, wavelength {:.0F} nm.".format(atom+state_str,
                                                                                                                max_index,
                                                                                                                (10*1000)/wave,
                                                                                                                wave*1000))
        absorption_dict[atom+state_str] = [wave,sec,hw,mass]

In [ ]:
# Make binned-down absorption dict.
bin_cross_dict = {}
for spec in tqdm(list(full_cross_dict.keys()),
                desc='Fixing cross sections...'):
    wave, sigma, mass = full_cross_dict[spec]
    wave = np.array([i for i in list(reversed(wave))])
    sigma = np.array([i for i in list(reversed(sigma))])

    # Get binned down version, where binning elements are 0.1 AA = 0.01 nm < 300 nm, and 2 nm > 300 nm.
    lam_step = 1e-5
    bin_w, bin_sig = [],[]
    lam0 = 0.1
    j = 0
    J = 0
    while lam0 < 1.0:
        ok = (wave>lam0) & (wave<lam0+lam_step)
        bin_w.append(np.mean(wave[ok]))
        bin_sig.append(np.mean(sigma[ok]))
        lam0 += lam_step
        if lam0 > 0.3:
            lam_step = 2e-3
        j += 1
        J += 1
        if j % 2000 == 0:
            print(spec, J, lam0, lam_step)
            j = 0

    bin_cross_dict[spec] = [np.array(bin_w),np.array(bin_sig),mass]

In [ ]:
### CONFIRMING I GOT WHAT I NEEDED AND SAVING OUT ###
for key in list(absorption_dict.keys()):
    wave, sig, hw, mass = absorption_dict[key]
    print('{}: peaks at {:.0F}+/-{:.3F} nm with cross-section {:.2E} cm^2'.format(key,1000*wave,1000*hw,sig))

# -peak will save only the biggest cross-section binned across some reasonable bandpass.
filename = 'cross-sections-peak.npy'
np.save(filename,absorption_dict)
# -full saves the entire native-res cross-section data.
filename = 'cross-sections-full.npy'
np.save(filename,full_cross_dict)
# -bin is the entire cross-section data binned down appropriately.
filename = 'cross-sections-bin.npy'
np.save(filename,bin_cross_dict)

This is the end of the four cells that generate the cross-sections. If you have already downloaded the Zenodo .npy cross-sections files, then you can skip ahead to this part.

In [ ]:
### LOADING AND PLOTTING ###
filename = 'cross-sections-peak.npy'
absorption_dict = np.load(filename,allow_pickle=True).item()

filename = 'cross-sections-full.npy'
full_cross_dict = np.load(filename,allow_pickle=True).item()

filename = 'cross-sections-bin.npy'
bin_cross_dict = np.load(filename,allow_pickle=True).item()

atoms = ('C','O','Na','S','K')
state_strs = ('_I','_II','_III')
colors = ('dodgerblue','red','lime','goldenrod','darkorchid')

plt.figure(figsize=(10,5))
for atom, color in zip(atoms,colors):
    for state_str, marker in zip(state_strs,('s','o','*')):
        try:
            lam, sigma, hws, mass = absorption_dict[atom+state_str]
            plt.errorbar(lam,sigma,xerr=hws,marker=marker,markersize=10,ls='none',label=atom+state_str,color=color)
        except KeyError:
            pass

plt.xlabel('wavelength [um]')
plt.ylabel('cross section [cm^2]')
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper right',ncols=5)
plt.xlim(0.1,1)
plt.xticks(ticks=[0.1,0.2,0.4,0.8,1.0],
           labels=['{:.1F}'.format(i) for i in [0.1,0.2,0.4,0.8,1.0]])
plt.ylim((1e-25,1e-12))
plt.show()
plt.close()

plt.figure(figsize=(10,5))
for atom, color in zip(atoms,colors):
    for state_str, ls in zip(state_strs,('-','--',':')):
        try:
            lam, sigma, mass = bin_cross_dict[atom+state_str]
            plt.plot(lam,sigma,label=atom+state_str,ls=ls,color=color)
        except KeyError:
            pass
for atom, color in zip(atoms,colors):
    for state_str, marker in zip(state_strs,('s','o','*')):
        try:
            lam, sigma, hws, mass = absorption_dict[atom+state_str]
            plt.errorbar(lam,sigma,xerr=hws,marker=marker,markersize=10,ls='none',markeredgecolor='k',color=color)
        except KeyError:
            pass

plt.xlabel('wavelength [um]')
plt.ylabel('cross section [cm^2]')
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper right',ncols=5)
plt.xlim(0.1,1)
plt.xticks(ticks=[0.1,0.2,0.4,0.8,1.0],#,10.0],
           labels=['{:.1F}'.format(i) for i in [0.1,0.2,0.4,0.8,1.0]])#,10.0]])
plt.ylim((1e-30,1e-10))
plt.show()
plt.close()

In [5]:
### CONSTANTS OF LENGTH ###
au      = 1.496e13  # cm
rads    = 6.957e10  # cm
rade    = 6.371e8   # cm

### CONSTANTS OF TIME ###
day     = 86400     # s
year    = 3.1536e7  # s

### CONSTANTS OF MASS ###
masss   = 1.989e33  # g
masse   = 5.9722e27 # g
amu     = 1.66e-24  # g

### OTHER CONSTANTS ###
G       = 6.6743e-8 # dyne cm^2 g^-2
kB      = 1.381e-16 # erg K^-1
pi      = np.pi     # no dims
c       = 2.9979e10 # cm s^-1
sigSB   = 5.67e-5   # erg cm^-2 s^-1 K^-4
mu0     = 1/(8*pi)  # g cm^-1 s^-2, cgs version of (1 Gauss)^2 / 2 mu0

### PLANETS WE ARE INTERESTED IN ###
tois = ['GJ 367 b', 'TOI-238 b', 'GJ 9827 b',
        'LHS 1678 b', 'L 168-9 b', 'K2-36 b',
        'HD 23472 d', 'GJ 486 b', 'HD 260655 b',
        'Kepler-102 b', 'GJ 9827 c', 'Kepler-102 c',
        'HD 63433 d']

best = ['GJ 367 b',]

### PLOTTING STUFF ###
fontsize_axis = 16
fontsize_tick = 14

In [6]:
### EQUATIONS ###
def torus_volume(a, H, n=3):
    # Torus volume assuming the torus spans n scale heights and starts at r_inner = a.
    r = n*H/2
    R = a + r
    return (pi*(r**2))*(2*pi*R)

def torus_mass(n, m, V):
    # Torus mass based on isotropic mass distribution.
    return n*m*V

def volc(tau, M):
    # Torus supply rate based on the torus mass and how long it takes to escape from the torus.
    return M/tau

def number_dens(fs, H, sig, n=3):
    # Number density which produces an absorption signature of fs in a torus of n scale heights for absorbers of size sig.
    ds = n*H
    return fs/(sig*ds)

def scale_height(T, m, Omega):
    # Plasma diffusion scale height, used here to approximate the scale radius.
    return np.sqrt((2/3)*kB*T/(m*(Omega**2)))

def get_teq(aoR,Teff):
    # 0-albedo equilibrium temperature, needed for scale height.
    return Teff*((1/4)**(1/4))/(aoR**0.5)

def lifetime(aoR, RA, Omega):
    # Minimum torus reservoir lifespan based on uninhibited magnetospheric convection.
    return np.sqrt(2/3)*((RA/aoR)**2)*(1/Omega)

def p_ratio(logB_mu, logB_sig, aoR, Teff, kapsig=1e-3):
    # The ratio of the magnetic pressure against radiation pressure, relevant to defining where radiation pressure may destroy a torus.
    prior_B = get_prior_B_p(logB_mu, logB_sig)
    post_prat = np.empty((100,))
    for i,b in enumerate(prior_B):
        mag_press = (b**2)*mu0*(aoR**-6)
        rad_press = (kapsig*sigSB*(Teff**4)/c)*(aoR**-2) #  kapsig is the estimated kappa*Sigma for a hot rarefied plasma, accounts for plasma transparency
        post_prat[i] = mag_press/rad_press
    return post_prat

def get_prior_B_p(logB_mu, logB_sig):
    prior_B = np.random.normal(logB_mu,logB_sig,100)
    prior_B = np.array([10**i for i in prior_B]) # in Gauss
    return prior_B

def vesc(m, r):
    # Escape velocity.
    return np.sqrt(2*G*m/r)

def eta_star(b, r, wind, vinf):
    # Eta*, the wind magnetic confinement parameter, which is dimensionless.
    return ((b**2)*mu0)*(r**2)/(wind*vinf)

def _residuals(ra, eta):
    # Residual function, see below.
    lhs = ra**4 - ra**3
    return (lhs-eta)**2

def get_alfven(eta):
    # Owocki 2009 Alfven radius solver.
    results = minimize(_residuals,np.array([20,]),args=(eta,))
    return results.x[0]

def alfven_posterior(star_radius, vinf, logB_mu, logB_sig,
                     logwind_mu, logwind_sig, N=1000):
    # Repeatedly solves the Owocki 2009 equation over a prior of B and wind values.
    prior_B, prior_wind = get_prior_B_wind(logB_mu, logB_sig, logwind_mu, logwind_sig, N)
    post_alfven = np.empty((N,N))
    for i, B in enumerate(prior_B):
        for j, w in enumerate(prior_wind):
            eta = eta_star(b=B,
                           r=rads*star_radius,
                           wind=w,
                           vinf=vinf)
            post_alfven[i,j] = get_alfven(eta)
    return post_alfven.flatten()

def get_prior_B_wind(logB_mu, logB_sig, logwind_mu, logwind_sig, N=1000):
    # returns the B and wind priors.
    prior_B = np.random.normal(logB_mu,logB_sig,N)
    prior_B = np.array([10**i for i in prior_B]) # in Gauss
    prior_wind = np.random.normal(logwind_mu,logwind_sig,N)
    prior_wind = np.array([(10**i)*masss/year for i in prior_wind]) # in g/s
    return prior_B, prior_wind

def empirical_mass(r):
    # The Chen & Kipping 2017 / Louie et al. 2018 mass-radius relations. Useful if you don't have a planet mass measurement on hand.
    emp_m = 0.9718*(r**3.58)
    if r > 1.23:
        emp_m = 1.436*(r**1.70)
    return emp_m

def get_tidal_power(p,rp,e):
    # Seligman et al. 2024 tidal power relation, scaled by the unknown quality factor.
    return (3.4*1e27)*(p**-5)*(rp**5)*((e/0.01)**2)

Some assumptions:
I approximate stellar magnetic field strengths and wind mass loss rates using a log-normal prior on B with a mean of 200 G and a width of 0.5 dex, and a Gaussian prior on the mass loss rate in solar masses per year centered on magnitude -14 with a width of 0.5 dex. This reflects the fact that most of the stars in the sample curated below are slow-rotating M and K stars with B and Mdot expected to be about 200 G and 1e-14 Msun/yr, with some wiggle to allow for the few fast rotators I have (e.g. TRAPPIST-1, HD 63433).

I also use the Chen & Kipping 2017 / Louie et al. 2018 mass-radius relation when the archive doesn't have a measured planetary mass. If you prefer not to estimate masses, you can always add an extra cut to the below rationale by filtering for targets with measured masses >0.

Rationale for how to pick the planets off of Exoplanet Archive (performed Nov 2024 so numbers may change when you try this):
To do these calculations, you have to have a stellar rotation rate, otherwise you will not be able to get a lifetime. So you need to enable that column, and set it to >0 so that you only get stars for which this is measured. This cuts the available targets down to 890. This is a big cut because not many people measure this value, since you normally don't need to know about it.

You are also going to need a known stellar mass, radius, and effective temperature, for getting the eta* parameter that helps you solve for the Alfven radius. Set all of those >0. Target N should fall to 795. These measurements are common so this cut is small.

You need to know the planet's semimajor axis in order to determine if it is inside the Alfven radius. So set orbit semimajor axis >0. The targets are now down to 617.

Since you are interested in outgassing from volcanically-active terrestrial planets, it won't benefit you to include large radii planets. Set planetary radius to bounds [0.0,1.60]. The number of targets is now 87. This is another big cut, because small planets are hard to find.

You could at this point cut to only incldue entries which are the default parameter set. If you do that, it drops to 41 available targets. I instead prefer to take all options, and then pick and choose between the planets that have duplicate entries.

At this stage, I take my targets. The columns I have are:

N planets in system

Whether it has been detected in RVs, which tells how good the ecc measurement is

Planet parameter reference, again useful for knowing whether the ecc measurement is good

Planet name

Planet orbit semimajor axis

Planet orbit period, needed if you are going to compute tidal heating rates

Planet radius

Planet mass

Planet eccentricity, which may be unknown

Planet equilibrium temperature, which may be unknown

Stellar radius

Stellar mass

Stellar effective temperature

Stellar rotation period

System V magnitude

I attached my table to the GitHub and Zenodo so you can quickly recreate this, but also, you should play around with it, see what else is out there. Some notes on special treatments I made:
1. LHS 1815 b is in my table as the Gan+ 2020 entry, but I used the better mass constraint from Luque+ 2022.
2. KOI 4777.01 has a weird mass measurement of <99 Earth masses on the archive, but their paper says it's meant to be <0.34 Earth masses. I think someone entered something wrong, so I replaced the data entry there with the correct upper bound.
3. Kepler-107 c makes it through all the checks but the mass measurements are really weird, like 2x iron density weird. I toss this target for safety's sake.
4. LHS 1678 b and c have two retrieved entries, Silverstein+ 2022 and 2024. I took the 2024 one since its ecc measurements had bounds, not just upper limits.
4. GJ 367 b retrieved two entries, Goffo+ 2023 and Lam+ 2021. Goffo+ 2023 is the default parameter set so I nabbed it.
5. GJ 486 b retrieved two entries, Trifonov+ 2021 and Caballero+ 2022. It did not collect the most recent set, Weiner Mansfield+ 2024, because that one didn't make a P_rot measurement for the star. I took most of the info from Weiner Mansfield+ 2024, subbing P_rot with Caballero+ 2022 data.
6. The GJ 9827 system had a plethora of measurements come up. Only Bonomo+ 2023 tried to estimate the eccentricity, so I took that one. It's only an upper bound though, so it's not particularly robust. But it is a multiplanetary system so that can be good for long-lived eccentricity.
7. K2-141 b had a Bonomo+ 2023 entry as well so I took that.
8. K2-36 b was the same as above.
9. The L 98-59 system was supplied entirely by Demangeon+ 2024.
10. LHS 1140 c retrieved Ment+ 2019 and Cadieux+ 2024. The latter was taken as it is the default parameter set.
11. For GJ 1132 b, I use the info from Xue+ 2024, except the stellar rotation which I get from Bonfils+ 2018.
12. For HD 93963 A b, I use the info from Polanski+ 2024, except the stellar rotation which I get from Serrano+ 2022.
13. For HIP 29442 b and c, I use the info from Egger+ 2024, except the stellar rotation which I get from Damasso+ 2023.
14. For K2-233 b and c, I use the info from Lillo-Box+ 2020, except the stellar rotation which I get from David+ 2018. Since they are both oddly massive by these studies, I let them go.
15. For the TOI-700 system, I use Gilbert+ 2023 for most parameters, and Gilbert+ 2020 for the stellar rotation.
16. For TRAPPIST-1, I use Agol+ 2021 for most parameters, Gillon+ 2016 for the stellar rotation, and Grimm+ 2018 for the eccentricity.

In [ ]:
# Read in our table.
data = pd.read_excel('plasma_planets3.xlsx',header=96)
print(list(data.keys())) # check that we got the header row right, this should say stuff like 'pl_name'

Now let's establish the workflow. We have x number of planets that we need to make some calculations for, as follows:
1. Compute vinf
2. Generate Alfven posterior
3. Generate tau posterior
4. Determine tidal heating, where applicable
5. Get the magnetic/radiation pressure ratio

Then, for each absorbing species and desired absorption strength:
1. Determine number density needed to produce absorption
2. Determine torus steady state mass with that number density
3. Determine torus mass input to sustain that steady state mass
4. Also get the column density, out of academic interest

In [ ]:
### STELLAR MAGNETOSPHERE PRIOR ###
logB_mu = 2.3
logB_sig = 0.5
logwind_mu = -14
logwind_sig = 0.5
posterior_root_samples = 100

# SHOW PRIORS FOR REFERENCE ###
prior_B, prior_wind = get_prior_B_wind(logB_mu, logB_sig, logwind_mu, logwind_sig, N=10000)
plt.figure(figsize=(int(20/3),8))
plt.hist(prior_B, bins=600,color='k',density=True)
plt.axvline(np.median(prior_B),color='red',ls='-')
plt.axvline(np.percentile(prior_B,25),color='red',ls='--')
plt.axvline(np.percentile(prior_B,75),color='red',ls='--')
plt.xlim(0,1000)
plt.xlabel(r'Stellar Magnetic Field [G]',fontsize=fontsize_axis)
plt.ylabel('Frequency',fontsize=fontsize_axis)
plt.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
plt.savefig('prior_b.png',
            dpi=300,bbox_inches='tight')
for perc in (25,50,75):
    print(np.percentile(prior_B,perc))
plt.show()
plt.close()

plt.figure(figsize=(int(20/3),8))
plt.hist(np.log10(prior_wind/(masss/year)),bins=30,color='k',density=True)
plt.axvline(np.median(np.log10(prior_wind/(masss/year))),color='red',ls='-')
plt.axvline(np.percentile(np.log10(prior_wind/(masss/year)),25),color='red',ls='--')
plt.axvline(np.percentile(np.log10(prior_wind/(masss/year)),75),color='red',ls='--')
plt.xlabel(r'Stellar Wind [log($\dot M_{wind}$ [M$_\odot$ year$^{-1}$])]',fontsize=fontsize_axis)
plt.xlim(-16,-12)
plt.ylabel('Frequency',fontsize=fontsize_axis)
plt.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
plt.savefig('prior_wind.png',
            dpi=300,bbox_inches='tight')
for perc in (25,50,75):
    print(np.percentile(np.log10(prior_wind/(masss/year)),perc))
plt.show()
plt.close()

vinf = vesc(m=masss*0.455,r=rads*0.458)
post_alf = alfven_posterior(0.5, vinf, logB_mu, logB_sig,
                            logwind_mu, logwind_sig, N=100)
plt.figure(figsize=(int(20/3),8))
plt.hist(post_alf,bins=75,color='k',density=True)
plt.axvline(np.median(post_alf),color='red',ls='-')
plt.axvline(np.percentile(post_alf,25),color='red',ls='--')
plt.axvline(np.percentile(post_alf,75),color='red',ls='--')
plt.xlabel('Alfven Radius [Stellar Radii]',fontsize=fontsize_axis)
plt.xlim(0,60)
plt.ylabel('Frequency',fontsize=fontsize_axis)
plt.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
plt.savefig('posterior_alfven.png',
            dpi=300,bbox_inches='tight')
for perc in (25,50,75):
    print(np.percentile(post_alf,perc))
plt.show()
plt.close()

In [8]:
### BIG WRAPPER FUNCTIONS ###
def process_planet(pdata,k,alfvens_star):
    # 0. Some useful numbers
    aor = (au/rads)*(pdata['pl_orbsmax'][k]/pdata['st_rad'][k])
    if pdata['pl_name'][k] == 'Io':
        aor = 6 # units for Io are a bit funky: R* is supposed to be Jupiter radius
    omega = 2*pi/(day*pdata['st_rotp'][k])
    Teff=pdata['st_teff'][k]
    teq = get_teq(aoR=aor,Teff=Teff)
    if pdata['pl_name'][k] == 'Io':
        teq = 110 # Io is orbiting Jupiter which is orbiting the Sun which really sets Teq
    ecc_err = [pdata['pl_orbeccenerr1'][k],pdata['pl_orbeccenerr2'][k]]
    V = pdata['sy_vmag'][k]
    starname = data['hostname'][k]

    # 1. Compute vinf
    vinf = vesc(m=masss*pdata['st_mass'][k],
                r=rads*pdata['st_rad'][k])
    
    # 2. Get Alfven posterior, mean, and bounds
    if starname in list(alfvens_star.keys()):
        # pull this info if it has already been gotten before
        alfven, alflo, alfhi = alfvens_star[starname]
    else:
        post_alfven = alfven_posterior(pdata['st_rad'][k],vinf,
                                    logB_mu=logB_mu,logB_sig=logB_sig,
                                    logwind_mu=logwind_mu,logwind_sig=logwind_sig,
                                    N=posterior_root_samples)
        alfven, alflo, alfhi = (np.percentile(post_alfven,50),
                                np.percentile(post_alfven,25),
                                np.percentile(post_alfven,75))

        alflo, alfven, alfhi = list(sorted([alflo,alfven,alfhi]))
        if pdata['pl_name'][k] == 'Io':
            alfven = 20
            alflo = 20
            alfhi = 20
        alfvens_star[starname] = [alfven, alflo, alfhi]

    # 3. Determine torus lifetime posterior
    tau = lifetime(aoR=aor,
                   RA=alfven,
                   Omega=omega)
    taulo = lifetime(aoR=aor,
                    RA=alflo,
                    Omega=omega)
    tauhi = lifetime(aoR=aor,
                    RA=alfhi,
                    Omega=omega)
    if pdata['pl_name'][k] == 'Io':
        tau = 50*day # 50 days is chosen since 20 to 80 days is the range of tau measured for Io and (20+80)/2 = 50
        taulo = 50*day
        tauhi = 50*day
    
    # 4. Determine tidal heating, where applicable
    tides = 0
    if all(not np.isnan(x) for x in (pdata['pl_orbper'][k],pdata['pl_rade'][k],pdata['pl_orbeccen'][k])):
        tides = get_tidal_power(pdata['pl_orbper'][k],pdata['pl_rade'][k],pdata['pl_orbeccen'][k])
        if np.isnan(tides):
            tides = 0
        if not np.isfinite(tides):
            tides = 0
    if pdata['pl_name'][k] == 'Io':
        tides = 1.6e21 # from Seligman+ 2024

    # 5. Get pressure ratio.
    post_prat = p_ratio(logB_mu=logB_mu,logB_sig=logB_sig,aoR=aor,Teff=Teff)
    prat, pratlo, prathi = (np.percentile(post_prat,50),
                            np.percentile(post_prat,25),
                            np.percentile(post_prat,75))
    if pdata['pl_name'][k] == 'Io':
        prat = 1 # dud
        pratlo = 1
        prathi = 1
    
    # Trim any planets that have weird masses
    pmass = pdata['pl_bmasse'][k]
    if (np.isnan(pmass) or pmass == 0):
        pmass = empirical_mass(r=pdata['pl_rade'][k])
    if pmass > 8.0:
        print(pdata['pl_name'][k],'had an abnormally huge mass ({} Earth masses with radius {} Earth radii) and was discarded.'.format(pmass,pdata['pl_rade'][k]))
        return False
    else:
        return alfven, alflo, alfhi, tau, taulo, tauhi, tides, aor, teq, V, pdata['pl_rade'][k], pmass, Teff, prat, pratlo, prathi, ecc_err, alfvens_star

def process_abs(pdata,tau,mol_mass,mol_sigma,
                fs=100e-6,n=3):
    # 0. Some useful numbers
    aor = pdata['aor']
    omega = 2*pi/(day*pdata['pstar'])
    Teff=pdata['Teff']
    teq = get_teq(aoR=aor,Teff=Teff)

    # 1. Determine number density needed to produce absorption
    h = scale_height(T=teq,m=amu*mol_mass,Omega=omega)
    n_dens = number_dens(fs,h,mol_sigma,n)

    # 2. Determine torus steady state mass with that number density
    V = torus_volume(au*pdata['a'],h,n)
    tMass = torus_mass(n_dens,amu*mol_mass,V)

    # 3. Determine torus mass input to sustain that steady state mass
    mdot = volc(tau,tMass)

    # 4. Get column density, out of academic interest
    ncol = n*h*n_dens
    
    return h, n_dens, tMass, mdot, ncol

The below cell will produce processed_output.npy, which can turn out slightly differently in each run due to the random draws from the B and Mdot priors when making the Alfven radius posteriors. If you want to replicate my plots exactly, just skip that cell. If you want to change anything (e.g. the B and Mdot priors), then make your edits and run the next cell.

In [ ]:
### COMPILING THESE NUMBERS TO MAKE PLOTS WITH ###
processed_output = {}
alfvens_star = {}
for k in tqdm(range(len(data)),
              desc='Processing planets...'):
    try:
        alfven, alflo, alfhi, tau, taulo, tauhi, tides, aor, teq, V, rp, mp, Teff, prat, pratlo, prathi, ecc_err, alfvens_star = process_planet(data,k,
                                                                                                                                                alfvens_star)
        if (alfven == alflo == alfhi and data['pl_name'][k] != 'Io'):
            print("we had a failed fit for alfven here!!", data['pl_name'][k])
        processed_output[data['pl_name'][k]] = {'alfven':[alfven,alflo,alfhi],
                                                'tau':[tau,taulo,tauhi],
                                                'tides':tides,
                                                'aor':aor,
                                                'teq':teq,
                                                'V':V,
                                                'rp':rp,
                                                'mp':mp,
                                                'Teff':Teff,
                                                'prat':[prat,pratlo,prathi],
                                                'ecc':data['pl_orbeccen'][k],
                                                'ecc_err':ecc_err,
                                                'a':data['pl_orbsmax'][k],
                                                'P':data['pl_orbper'][k],
                                                'mstar':data['st_mass'][k],
                                                'rstar':data['st_rad'][k],
                                                'pstar':data['st_rotp'][k],
                                                'dist':data['sy_dist'][k],
                                                'starname':data['hostname'][k]
                                                }
        print(data['pl_name'][k],processed_output[data['pl_name'][k]])
    except TypeError:
        pass
filename = 'processed-output.npy'
np.save(filename,processed_output)

In [19]:
### PLOTTING FUNCTIONS ###
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap
cmap = plt.get_cmap('RdYlBu')
spectral_cmap = truncate_colormap(cmap, 0.0, 0.55)

def plot_alfven_aor(names,alfvens,alflos,alfhis,aors,rps,Tstar):
    # Rescale size
    F = 1000
    sizes = F*np.log10(np.square(np.array(rps))+1)
    re_size = F*np.log10(np.square(np.array(1))+1)
    
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))

    ax.plot([0,50],[0,50],color='grey',ls='--',zorder=0)
    ax.fill_between(x=[0,50],y1=[0,50],y2=[0,0],color='grey',alpha=0.5)

    lower = [alfv - alfl for alfv, alfl in zip(alfvens,alflos)]
    upper = [alfh - alfv for alfh, alfv in zip(alfhis,alfvens)]
    ax.errorbar(aors,alfvens,yerr=[lower,upper],c='k',fmt='none',ls='none',capsize=3,zorder=1)
    im = ax.scatter(aors,alfvens,c=Tstar,cmap=spectral_cmap,edgecolors='k',s=sizes,vmin=2500,vmax=6000)
    
    
    for name, alfven, aor, size in zip(names,alfvens,aors,sizes):
        if (all(i <= 40 for i in (alfven,aor))):# and name=='Io'):
            if name in tois:
                ax.scatter(aor,alfven,s=size,color='None',edgecolor='blue',lw=3)
            elif name == 'Io':
                ax.scatter(aor,alfven,s=size,color='black',edgecolor='black')
                ax.text(aor,alfven-1,s=name,
                        ha='center',va='center',fontsize=fontsize_tick,
                        color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                              foreground='white')])
            if name in best:
                ax.scatter(aor,alfven,s=size,color='blue',marker='x',lw=1)
    
    im2 = ax.scatter(-1000,-1000,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(-1000,-1000,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='upper right',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)
    
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower right',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(0,30)
    plt.ylim(0,30)
    plt.xticks(ticks=[0,5,10,15,20,25,30])
    plt.yticks(ticks=[0,5,10,15,20,25,30])
    ax.set_xlabel('Semimajor Axis [Stellar Radii]',fontsize=fontsize_axis)
    ax.set_ylabel('Alfven Radius [Stellar Radii]',fontsize=fontsize_axis)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    cb = plt.colorbar(mappable=im,)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)

    plt.savefig('alfven_radii.png',dpi=300,bbox_inches='tight')
    plt.close()

def plot_pressure_aor(names,prats,pratlos,prathis,alfvens,aors,rps,Tstar):
    # Rescale size
    F = 1000
    sizes = F*np.log10(np.square(np.array(rps))+1)
    re_size = F*np.log10(np.square(np.array(1))+1)
    
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))

    ax.axhline(y=100,ls='--',color='grey',zorder=0)
    ax.fill_between(x=[0,25],y1=[100,100],y2=[1e-10,1e-10],color='grey',alpha=0.5)

    lols = np.array([i/j for i,j in zip(aors,alfvens)])
    aors = np.array(aors)
    prats = np.array(prats)
    Tstar = np.array(Tstar)
    sizes = np.array(sizes)
    names = np.array(names)

    lower = np.array([prat - plo for prat, plo in zip(prats,pratlos)])[lols<=1]
    upper = np.array([phi - prat for phi, prat in zip(prathis,prats)])[lols<=1]
    ax.errorbar(aors[lols<=1],prats[lols<=1],yerr=[lower,upper],c='k',fmt='none',ls='none',capsize=3,zorder=1)

    im = ax.scatter(aors[lols<=1],prats[lols<=1],c=Tstar[lols<=1],cmap=spectral_cmap,edgecolors='k',s=sizes[lols<=1],vmin=2500,vmax=6000)
    
    for name, prat, aor, size in zip(names[lols<=1],prats[lols<=1],aors[lols<=1],sizes[lols<=1]):
        if aor != 0:
            if name in tois:
                ax.scatter(aor,prat,s=size,color='None',edgecolor='blue',lw=3)
            elif name == 'Io':
                ax.scatter(aor,prat,s=size,color='white',edgecolor='white')
            if name in best:
                ax.scatter(aor,prat,s=size,color='blue',marker='x',lw=1)
    
    im2 = ax.scatter(1000,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1000,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='upper right',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)
    
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower left',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(0,25)
    plt.ylim(1,1e6)
    plt.yscale('log')
    plt.xticks(ticks=[0,5,10,15,20,25])
    ax.set_xlabel('Semimajor Axis [Stellar Radii]',fontsize=fontsize_axis)
    ax.set_ylabel(r'$\zeta$ [Magnetic Pressure/Radiation Pressure]',fontsize=fontsize_axis)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    cb = plt.colorbar(mappable=im,)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)

    plt.savefig('pressure_ratios.png',dpi=300,bbox_inches='tight')
    plt.close()

def plot_timescale(taus,taulos,tauhis,alfvens,alflos,alfhis,aors,names,rps,Tstar):
    # Rescale size
    F = 1000
    re_size = F*np.log10(np.square(np.array(1))+1)
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))
    lols = np.array([i/j for i,j in zip(aors,alfvens)])
    lollos = np.array([i/j for i,j in zip(aors,alflos)])
    lolhis = np.array([i/j for i,j in zip(aors,alfhis)])
    lower_lol = np.array([x - y for x, y in zip(lols,lolhis)])
    upper_lol = np.array([x - y for x, y in zip(lollos,lols)])
    
    staus = np.array([t/day for t in taus])
    staulos = np.array([t/day for t in taulos])
    stauhis = np.array([t/day for t in tauhis])
    lower_stau = np.array([x - y for x, y in zip(staus,staulos)])
    upper_stau = np.array([x - y for x, y in zip(stauhis,staus)])
    
    Tstar = np.array(Tstar)

    ax.errorbar(lols[lols<=1],staus[lols<=1],xerr=[lower_lol[lols<=1],upper_lol[lols<=1]],yerr=[lower_stau[lols<=1],upper_stau[lols<=1]],c='k',fmt='none',ls='none',capsize=3,zorder=1)
    im = ax.scatter(lols[lols <= 1],staus[lols <= 1],c=Tstar[lols <= 1],cmap=spectral_cmap,edgecolors='k',s=sizes[lols <= 1],vmin=2500,vmax=6000)
    for name, lol, tau, size in zip(names,lols,staus,sizes):
        if (lol <= 1): # and name=='Io'):
            if name in tois:
                ax.scatter(lol,tau,s=size,color='None',edgecolor='blue',lw=3)
            elif name == 'Io':
                ax.scatter(lol,tau,s=size,color='black',edgecolor='black')
                ax.text(lol,tau-10,s=name,
                        ha='center',va='center',fontsize=fontsize_tick,
                        color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                              foreground='white')])
            if name in best:
                ax.scatter(lol,tau,s=size,color='blue',marker='x',lw=1)
    
    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower left',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)

    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='upper right',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(0,1)
    plt.yscale('log')
    plt.ylim(0.5,500)
    plt.xticks(ticks=[0,0.25,0.5,0.75,1.00])
    plt.yticks(ticks=[1,10,100,500],
               labels=['1.0','10.0','100.0','500.0'])
    ax.set_xlabel('Semimajor Axis [Alfven Radii]',fontsize=fontsize_axis)
    ax.set_ylabel('Magnetospheric Convection Timescale [Days]',fontsize=fontsize_axis)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)

    plt.savefig('alfven_tau.png',dpi=300,bbox_inches='tight')
    plt.close()

def plot_tau_tides(names,taus,taulos,tauhis,masses,tides,alfvens,aors,rps,v):
    zl = 'Magnitude [V]'
    C = 'Greens'
    vmin, vmax = (5,13)
    
    # Rescale size
    F = 1000
    re_size = F*np.log10(np.square(np.array(1))+1)
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))
    lols = np.array([i/j for i,j in zip(aors,alfvens)])
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))[lols <= 1]
    stides = np.array([t/(masse*m) for t,m in zip(tides,masses)])[lols <= 1]
    staus = np.array([t/day for t in taus])[lols <= 1]
    staulos = np.array([t/day for t in taulos])[lols <= 1]
    stauhis = np.array([t/day for t in tauhis])[lols <= 1]
    lower_stau = np.array([x - y for x, y in zip(staus,staulos)])
    upper_stau = np.array([x - y for x, y in zip(stauhis,staus)])
    v = np.array(v)[lols <= 1]
    snames=np.array(names)[lols <= 1]

    # Make the Io point invisible
    io_index = np.argwhere(snames=='Io')[0]
    original_io_size = sizes[io_index]
    sizes[io_index] = 0

    ax.errorbar(stides[stides > 0],staus[stides > 0],yerr=[lower_stau[stides > 0],upper_stau[stides > 0]],c='k',fmt='none',ls='none',capsize=3,zorder=1)
    im = ax.scatter(stides[stides > 0],staus[stides > 0],c=v[stides > 0],cmap=C,edgecolors='blue',s=sizes[stides > 0],
                    vmin=vmin,vmax=vmax,lw=3)
    for name, tide, tau, size in zip(snames[stides > 0],stides[stides > 0],staus[stides > 0],sizes[stides > 0]):
        if name == 'Io':
            ax.scatter(tide,tau,s=original_io_size,c=5,cmap=C,edgecolor='black')
            ax.text(tide,tau-15,s=name,
                    ha='center',va='center',fontsize=fontsize_tick,
                    color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                            foreground='white')])
        if name in best:
            ax.scatter(tide,tau,s=size,color='None',edgecolor='blue',lw=3)
            ax.scatter(tide,tau,s=size,color='blue',marker='x',lw=1)

    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='upper left',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)
            
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower left',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(1e-5,1e4)
    plt.xticks([1e-5,1e-3,1e-1,1e1,1e3])
    plt.xscale('log')
    plt.yscale('log')
    plt.ylim(0.5,100)
    y_ticks = [1,5,10,50,100,500]
    plt.yticks(ticks=y_ticks,
            labels=['%.1f' % x for x in y_ticks])
    plt.rcParams['mathtext.default'] = 'regular'
    ax.set_xlabel(r'$\dot{E}_{\rm heat}$/Im(k$_2$)/Mass [erg/s/g]',fontsize=fontsize_axis)
    ax.set_ylabel('Magnetospheric Convection Timescale [Days]',fontsize=fontsize_axis)
    cb = plt.colorbar(mappable=im)
    cb.set_label(zl, fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    plt.savefig('alfven_tides.png',dpi=300,bbox_inches='tight')
    plt.close()

def generate_table_lifetimes(names,a,ra,ra_l,ra_h,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,taus,tau_l,tau_h,ecc_errs,dists,prats):
    # Sort everything by torus lifetime.
    b = [k for k in zip(names,a,ra,ra_l,ra_h,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,ecc_errs,dists,prats,tau_l,tau_h,taus) if not np.isnan(k[-1])]
    b = list(reversed(sorted(b,key=lambda x:x[-1])))
    names,a,ra,ra_l,ra_h,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,ecc_errs,dists,prats,tau_l,tau_h,taus = [[x[i] for x in b] for i in range(len(b[0]))]

    # Convert alfven radius to AU.
    ra = [x*rs*rads/au for x,rs in zip(ra,rstar)]
    ra_l = [x*rs*rads/au for x,rs in zip(ra_l,rstar)]
    ra_l = [x-y for x, y in zip(ra,ra_l)]
    ra_h = [x*rs*rads/au for x,rs in zip(ra_h,rstar)]
    ra_h = [x-y for x, y in zip(ra_h,ra)]
    # Convert timescale to days.
    taus = [x/day for x in taus]
    tau_l = [x/day for x in tau_l]
    tau_l = [x-y for x, y in zip(taus,tau_l)]
    tau_h = [x/day for x in tau_h]
    tau_h = [x-y for x, y in zip(tau_h,taus)]
    # Get H for 32 amu.
    H = []
    for i in range(len(names)):
        H.append(scale_height(teq[i],32*amu,2*pi/(pstar[i]*day)))

    # Round prat.
    prat_new = []
    for p in prats:
        if p > 1:
            prat_new.append(np.round(p,decimals=0))
        elif p > 0.1:
            prat_new.append(np.round(p,decimals=1))
        else:
            prat_new.append(np.round(p,decimals=2))
    prats = prat_new

    # Fill out table.tex file.
    with open('table.tex',mode='w') as f:
        for i in range(len(names)):
            if names[i] == "Io":
                continue
            if (tides[i] > 0 and a[i]/ra[i] <= 1):
                if np.isnan(ecc_errs[i][0]):
                    f.write('{} & {:.3F} & {:.3F}$^{{+{:.3F}}}_{{-{:.3f}}}$ & {:.2F} & {} & {:.3F} & $<${:.3F} & {:.3F} & {:.0F} & {:.2F} & {:.2F} & {:.2F} & {:.3F} & {:.0F} & {:.2E} & {:.0F}$^{{+{:.0F}}}_{{-{:.0f}}}$ & \\\\ \n'.format(names[i],
                                            a[i],
                                            ra[i],
                                            ra_h[i],
                                            ra_l[i],
                                            H[i]/(100*1000*1e5), # turn cm into 10^5 km
                                            prats[i],
                                            P[i],
                                            ecc[i],
                                            rp[i],
                                            teq[i],
                                            mstar[i],
                                            rstar[i],
                                            pstar[i],
                                            vmag[i],
                                            dists[i],
                                            tides[i],
                                            taus[i],
                                            tau_h[i],
                                            tau_l[i]))
                else:
                    f.write('{} & {:.3F} & {:.3F}$^{{+{:.3F}}}_{{-{:.3f}}}$ & {:.2F} & {} & {:.3F} & {:.3F}$^{{+{:.3F}}}_{{{:.3f}}}$ & {:.3F} & {:.0F} & {:.2F} & {:.2F} & {:.2F} & {:.3F} & {:.0F} & {:.2E} & {:.0F}$^{{+{:.0F}}}_{{-{:.0f}}}$ & \\\\ \n'.format(names[i],
                                            a[i],
                                            ra[i],
                                            ra_h[i],
                                            ra_l[i],
                                            H[i]/(100*1000*1e5), # turn cm into 10^5 km
                                            prats[i],
                                            P[i],
                                            ecc[i],
                                            ecc_errs[i][0],
                                            ecc_errs[i][1],
                                            rp[i],
                                            teq[i],
                                            mstar[i],
                                            rstar[i],
                                            pstar[i],
                                            vmag[i],
                                            dists[i],
                                            tides[i],
                                            taus[i],
                                            tau_h[i],
                                            tau_l[i]))
                
def generate_table_outgassing(names,mdots,taus,tmasses):
    # Sort everything by torus lifetime.
    b = [k for k in zip(names,mdots,tmasses,taus)]
    b = list(reversed(sorted(b,key=lambda x:x[-1])))
    names,mdots,tmasses,taus = [[x[i] for x in b] for i in range(len(b[0]))]
    
    # Find total outgassing rate of atoms and molecules in ton/s, masking those which are too large.
    total_atoms = []
    for m in mdots:
        mass = np.sum([mi for mi in m if mi < 1e10]) # catches for any of the five species below which need way too much outgassing to be seen
        total_atoms.append(mass/1e6)
    total_atoms = np.array(total_atoms)

    # Get quasi-steady state mass in Mton.
    torus_mass = []
    for m in tmasses:
        mass = np.sum([mi for mi in m if mi < 1e16]) # catches for any of the five species below which need way too much outgassing to be seen
        torus_mass.append(mass/1e12)
    torus_mass = np.array(torus_mass)
    

    with open('table2.tex',mode='w') as f:
        top_str = ''
        for spec in list(absorption_dict.keys()):
            if spec in ('C_III','O_II','O_III','Na_II','K_II'):
                  # Hard-coding to pass on these because they have cross sections under 1e-16 cm2 and therefore
                  # it would be much too hard with present instrumentation to detect their sigs.
                  continue
            else:
                top_str += '{} & '.format(spec)
        top_str += '\n'
        f.write(top_str)
        for i in range(len(names)):
            if names[i] == "Io":
                continue
            mdot_planet = [k/1e6 for k in mdots[i]]
            mdot_planet = ["{:.0F}".format(f) if f>10 else "{:.1F}".format(f) for f in mdot_planet]
            mdot_string = ""
            for mdot in mdot_planet:
                if float(mdot) <= 1e10:
                    mdot_string += '{} & '.format(mdot)
                else:
                    pass
            if total_atoms[i] > 10:
                total_atoms_planet = int(total_atoms[i])
            else:
                total_atoms_planet = np.round(total_atoms[i],decimals=1)
            if torus_mass[i] > 10:
                torus_mass_planet = int(torus_mass[i])
            else:
                torus_mass_planet = np.round(torus_mass[i],decimals=1)
            f.write('{} & {}{} & {} \\\\ \n'.format(names[i],
                                                    mdot_string,
                                                    total_atoms_planet,
                                                    torus_mass_planet))

In [ ]:
# Generate all plots and make the table.
processed_output = np.load('processed-output.npy',allow_pickle=True).item()

plot_alfven_aor(names=list(processed_output.keys()),
                alfvens=[x[0] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                alflos=[x[1] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                alfhis=[x[2] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                aors=[x for x in [processed_output[key]['aor'] for key in list(processed_output.keys())]],
                rps=[x for x in [processed_output[key]['rp'] for key in list(processed_output.keys())]],
                Tstar=[x for x in [processed_output[key]['Teff'] for key in list(processed_output.keys())]])

plot_pressure_aor(names=[key for key in list(processed_output.keys()) if key != "Io"],
                  prats=[x[0] for x in [processed_output[key]['prat'] for key in list(processed_output.keys()) if key != "Io"]],
                  pratlos=[x[1] for x in [processed_output[key]['prat'] for key in list(processed_output.keys()) if key != "Io"]],
                  prathis=[x[2] for x in [processed_output[key]['prat'] for key in list(processed_output.keys()) if key != "Io"]],
                  alfvens=[x[0] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys()) if key != "Io"]],
                  aors=[x for x in [processed_output[key]['aor'] for key in list(processed_output.keys()) if key != "Io"]],
                  rps=[x for x in [processed_output[key]['rp'] for key in list(processed_output.keys()) if key != "Io"]],
                  Tstar=[x for x in [processed_output[key]['Teff'] for key in list(processed_output.keys()) if key != "Io"]])

plot_timescale(taus=[x[0] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                taulos=[x[1] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                tauhis=[x[2] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                alfvens=[x[0] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                alflos=[x[1] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                alfhis=[x[2] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                aors=[x for x in [processed_output[key]['aor'] for key in list(processed_output.keys())]],
                names=list(processed_output.keys()),
                rps=[x for x in [processed_output[key]['rp'] for key in list(processed_output.keys())]],
                Tstar=[x for x in [processed_output[key]['Teff'] for key in list(processed_output.keys())]])

plot_tau_tides(names=list(processed_output.keys()),
               taus=[x[0] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                taulos=[x[1] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                tauhis=[x[2] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                masses=[x for x in [processed_output[key]['mp'] for key in list(processed_output.keys())]],
                tides=[x for x in [processed_output[key]['tides'] for key in list(processed_output.keys())]],
                alfvens=[x[0] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                aors=[x for x in [processed_output[key]['aor'] for key in list(processed_output.keys())]],
                rps=[x for x in [processed_output[key]['rp'] for key in list(processed_output.keys())]],
                v=[x for x in [processed_output[key]['V'] for key in list(processed_output.keys())]])

generate_table_lifetimes(names=list(processed_output.keys()),
                         a=[x for x in [processed_output[key]['a'] for key in list(processed_output.keys())]],
                         ra=[x[0] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                        ra_l=[x[1] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                        ra_h=[x[2] for x in [processed_output[key]['alfven'] for key in list(processed_output.keys())]],
                         P=[x for x in [processed_output[key]['P'] for key in list(processed_output.keys())]],
                         ecc=[x for x in [processed_output[key]['ecc'] for key in list(processed_output.keys())]],
                         rp=[x for x in [processed_output[key]['rp'] for key in list(processed_output.keys())]],
                         teq=[x for x in [processed_output[key]['teq'] for key in list(processed_output.keys())]],
                         mstar=[x for x in [processed_output[key]['mstar'] for key in list(processed_output.keys())]],
                         rstar=[x for x in [processed_output[key]['rstar'] for key in list(processed_output.keys())]],
                         pstar=[x for x in [processed_output[key]['pstar'] for key in list(processed_output.keys())]],
                         vmag=[x for x in [processed_output[key]['V'] for key in list(processed_output.keys())]],
                         tides=[x for x in [processed_output[key]['tides'] for key in list(processed_output.keys())]],
                         taus=[x[0] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                         tau_l=[x[1] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                         tau_h=[x[2] for x in [processed_output[key]['tau'] for key in list(processed_output.keys())]],
                         ecc_errs=[x for x in [processed_output[key]['ecc_err'] for key in list(processed_output.keys())]],
                         dists=[x for x in [processed_output[key]['dist'] for key in list(processed_output.keys())]],
                         prats=[x[0] for x in [processed_output[key]['prat'] for key in list(processed_output.keys())]])

In [ ]:
# Use the absorption dict, which contains [central lambda,cross-section,halfwidth,mass], to get n_dens needed for 10% absorption
hs, n_denses, tMasses, mdots, ncols = [],[],[],[],[]
new_names = []
new_taus = []
for name in list(processed_output.keys()):
    print('------------')
    try:
        if name == 'Io':
            continue # not trying to do exoplanetary science on Io, for obvious reasons
        if processed_output[name]['alfven'][0] < processed_output[name]['aor']:
            print(1/0) # zero division error lets me catch targets tossed for being outside the Alfven radius
        if processed_output[name]['tides'] == 0:
            print(flop) # undefined variable lets me catch targets tossed for no tidal heating
        hk, nk, tk, mk, nck = [],[],[],[],[]
        mol_sigs_check  =[]
        specs_done = []
        for spec in list(absorption_dict.keys()):
            central_lams,mol_sigma,halfwidths,mol_mass = absorption_dict[spec]
            if mol_sigma <= 1e-16:
                # Hard-coding to skip species that do not have enough cross-section for detection.
                # You can of course delete this condition and see the billion ton/s rates needed to
                # produce detectable volcanism with present instrumentation.
                continue
            mol_sigs_check.append(mol_sigma)
            
            F = 0.10 # 10% absorption

            h_i, n_dens_i, tMass_i, mdot_i, ncol_i = process_abs(processed_output[name],processed_output[name]['tau'][1],
                                                                 mol_mass,mol_sigma,fs=F,n=3)
            
            print(spec,": {:.0f} cm^-3".format(n_dens_i),"{:.2E} cm^2".format(3*n_dens_i*h_i),"{:.2F} ton/s".format(mdot_i/1e6))

            h_i, n_dens_i, tMass_i, mdot_i, ncol_i = process_abs(processed_output[name],processed_output[name]['tau'][2],
                                                                 mol_mass,mol_sigma,fs=F,n=3)
            
            print(spec,": {:.0f} cm^-3".format(n_dens_i),"{:.2E} cm^2".format(3*n_dens_i*h_i),"{:.2F} ton/s".format(mdot_i/1e6))
            
            h_i, n_dens_i, tMass_i, mdot_i, ncol_i = process_abs(processed_output[name],processed_output[name]['tau'][0],
                                                                 mol_mass,mol_sigma,fs=F,n=3)
            
            print(spec,": {:.0f} cm^-3".format(n_dens_i),"{:.2E} cm^2".format(3*n_dens_i*h_i),"{:.2F} ton/s".format(mdot_i/1e6))
            hk.append(h_i)
            nk.append(n_dens_i)
            tk.append(tMass_i)
            mk.append(mdot_i)
            nck.append(ncol_i)
            specs_done.append(spec)
        hs.append(hk)
        n_denses.append(nk)
        tMasses.append(tk)
        mdots.append(mk)
        ncols.append(nck)
        new_names.append(name)
        new_taus.append(processed_output[name]['tau'])
        print(name,':',np.round(processed_output[name]['tau'][0]/86400,decimals=0),'days')
        for spec, t, m, nds, sig in zip(specs_done,tk,mk,nck,mol_sigs_check):
            print(spec,'{:.2F} MT'.format(t/1e12),'{:.2F} ton/s'.format(m/1e6),'{:.0F}% abs.'.format(100*nds*sig))
        print('total outgassing:',np.round(np.sum(mk)/1e6,decimals=0),'ton/s')
        print('total mass:',np.round(np.sum(tk)/1e12,decimals=0),'MT')
        print('expected mass:',np.round((np.sum(mk)/1e12)*processed_output[name]['tau'][0],decimals=0),'MT')
    except ZeroDivisionError:
        print("passing on ",name,'for bad aoR')
        pass
    except NameError:
        print("passing on ",name,'for no tides')
    except IndexError:
        print("ran out of indices")
generate_table_outgassing(new_names,mdots,new_taus,tMasses)

Lastly, we need a plot of the stellar spectrum of GJ 367 with and without torus contamination.

In [ ]:
M, Teff, logg = (-0.01, 3522, 4.77) # Goffo+ 2023 stellar parameters

print([key for key in list(absorption_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')])

# Define the binned wavelength scheme
bin_waves1 = np.arange(0.1,0.3,1e-5)
bin_waves2 = np.arange(0.3,1.0,2e-3)
bin_waves = np.concatenate((bin_waves1,bin_waves2))

wave, sigma, mass = full_cross_dict['S_I']
bin_waves1 = np.arange(np.min(wave),0.3,1e-5)
bin_waves2 = np.arange(0.3,1.00,2e-3)
bin_waves = np.concatenate((bin_waves1,bin_waves2))
wave = np.array([i for i in list(reversed(wave))])
ok = (wave>0.1) & (wave<1.0)
bin_waves = wave[ok]
print(bin_waves)

# Open PEGASUS SED
with fits.open('pegasus_spectrum/GJ367_PHOENIX_model.fits') as sp2:
    wave = sp2[1].data['Wavelength']*1e-4 # angstrom to um is 1e-4
    flux = sp2[1].data['Flux_Density']*1.204e-18 # scale from stellar surface to distance to observer

# Truncate flux to our range of interest.
ok = (wave>0.1) & (wave<1.00)
fluxGJ = flux[ok]
waveGJ = wave[ok]

plt.figure(figsize=(20,5))
plt.plot(waveGJ,fluxGJ)
plt.xlabel('wavelength [um]')
plt.ylabel(r'flux [erg/s/cm$^2$/$\AA$]')
plt.xlim(0.1,0.3)
plt.yscale('log')
#plt.ylim(1e-,1)
plt.show()
plt.close()

# For each species, need to get its scale height
# And we need to interpolate the MUSCLES SED onto this grid
from scipy.interpolate import interp1d
interpolated = False
full_cross_dict_H = {}
bin_cross_dict_H = {}

colors = {'C':'dodgerblue',
          'O':'red',
          'N':'lime',
          'S':'goldenrod',
          'K':'darkorchid'}
plotting_dict = {}
for spec in list(full_cross_dict.keys()):
    color = colors[spec[0]]
    if '_III' in spec:
        ls = ':'
    elif '_II' in spec:
        ls = '--'
    else:
        ls = '-'
    plotting_dict[spec] = (color,ls)

plt.figure(figsize=(20,5))
for spec in tqdm(list(full_cross_dict.keys()),
                desc='Fixing cross sections...'):
    wave, sigma, mass = full_cross_dict[spec]
    wave = np.array([i for i in list(reversed(wave))])
    sigma = np.array([i for i in list(reversed(sigma))])

    interpolater = interp1d(wave,sigma,fill_value=0,bounds_error=False)
    new_sigma = interpolater(bin_waves)
    wave = bin_waves
    sigma = new_sigma
    print(wave.shape)

    h = scale_height(T=1368,m=mass*amu,Omega=2*pi/(day*51.3)) # 1368 K is eq temp, 51.3 d is rotation period

    full_cross_dict_H[spec] = [wave,sigma,h]
    
    if not interpolated:
        interpolater = interp1d(waveGJ,fluxGJ,fill_value=0,bounds_error=False)
        new_flux = interpolater(bin_waves)
        waveGJ = bin_waves
        fluxGJ = new_flux
        interpolated = True
        print(waveGJ.shape)
    
    wave, sigma, mass = bin_cross_dict[spec]
    bin_cross_dict_H[spec] = [wave,sigma,h]
    print(wave.shape)

    plt.plot(wave,sigma,label=spec,color=plotting_dict[spec][0],ls=plotting_dict[spec][1])
plt.legend()
plt.xlabel('wavelength [um]')
plt.ylabel('cross-section [cm2]')
plt.yscale('log')
plt.xlim(0.10, 1.00)
plt.ylim(1e-25,1e-10)
plt.show()
plt.close()

# Recall that f = Ncol ds = n sigma 3 H
# We have h for each species and sigma for each wavelength for each species
# n was determined in the cell above, let's go get it
for n_dens, name in zip(n_denses, new_names):
    if name == 'GJ 367 b':
        number_densities = n_dens
ndens_dict = {}
for i, key in enumerate([key for key in list(absorption_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')]):
    ndens_dict[key] = number_densities[i]

# Now that we have gotten n for each species, we can get f.
plt.figure(figsize=(10,5))
for spec in tqdm([key for key in list(absorption_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')],
                desc='Computing absorption...'):
    wave, sigma, h = full_cross_dict_H[spec]
    print(wave,np.max(sigma))
    plt.plot(wave,sigma,label=spec,color=plotting_dict[spec][0],ls=plotting_dict[spec][1],alpha=0.5)

    wave, sigma, h = bin_cross_dict_H[spec]
    print(wave,np.max(sigma))
    plt.plot(wave,sigma,label=spec,color=plotting_dict[spec][0],ls=plotting_dict[spec][1],alpha=0.5)

    central_lams,mol_sigma,halfwidths,mol_mass = absorption_dict[spec]
    plt.scatter(central_lams,mol_sigma,label=spec,color=plotting_dict[spec][0],ls=plotting_dict[spec][1],edgecolors='k')
    
plt.legend()
plt.xlabel('wavelength [um]')
plt.ylabel('cross-section [cm2]')
plt.yscale('log')
plt.xlim(0.10, 1.00)
plt.ylim(1e-25,1e-10)
plt.show()
plt.close()

modified_flux = np.copy(fluxGJ)
mod_fluxes = []
plt.figure(figsize=(10,5))
for spec in tqdm([key for key in list(absorption_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')],
                desc='Computing absorption...'):
    wave, sigma, h = full_cross_dict_H[spec]
    n_dens = ndens_dict[spec]
    print(spec,h,n_dens,np.max(n_dens*3*sigma*h))
    f = 1 - (n_dens * sigma * 3 * h)
    print(np.min(f),np.max(f))
    f[f<0] = 0
    print(spec,np.min(f),np.max(f))
    mod_fluxes.append(modified_flux*f)

    wave, sigma, h = bin_cross_dict_H[spec]
    bin_f = (n_dens * sigma * 3 * h)
    bin_f[bin_f<0] = 0
    print(spec,np.min(bin_f),np.max(bin_f))

    plt.plot(wave,100*bin_f,label=spec,color=plotting_dict[spec][0],ls=plotting_dict[spec][1]) # plot the absorption of each species by wavelength
plt.legend()
plt.xlabel('wavelength [um]')
plt.ylabel('absorption [%]')
plt.xscale('log')
plt.xlim(0.10, 1.00)
plt.ylim(0.0,15.0)
plt.show()
plt.close()


fig, ax = plt.subplots(figsize=(20,5))
for mod_flux,spec in zip(mod_fluxes,
                         [key for key in list(absorption_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')]):
    ax.plot(waveGJ,mod_flux,color=plotting_dict[spec][0],ls=plotting_dict[spec][1],label=spec)
ax.plot(waveGJ,fluxGJ,ls='-',color='k',alpha=0.75)
ax.legend(loc='lower right',fontsize=fontsize_tick)

# Making the plot pretty
ax.set_yscale('log')

ax.set_xscale('log')
ax.set_xlim(0.1,1.0)
xt = [0.1,0.2,0.3,0.4,0.5,0.75,1.00]
ax.set_xticks(xt)
ax.set_xticklabels(['{:.2F}'.format(f) for f in xt])

from matplotlib.ticker import NullFormatter as nf
ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax.xaxis.set_minor_formatter(nf())
ax.set_xlabel(r'Wavelength [$\mu$m]',fontsize=fontsize_axis)
ax.set_ylabel((r'Flux [erg/s/cm$^2$/$\AA$]'),fontsize=fontsize_axis)
plt.rc('grid',color='grey',alpha=0.5,ls=':')
plt.grid()
plt.show()
plt.close()

In [ ]:
# Now that we have gotten n for each species, we can get f.
from matplotlib.ticker import NullFormatter as nf

absorptions = {}
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(20,10),sharex=True)
plt.subplots_adjust(hspace=0.05)
for mod_flux,spec in zip(mod_fluxes,[key for key in list(full_cross_dict.keys()) if key not in ('C_III','O_II','O_III','Na_II','K_II')]):
    ax[0].plot(waveGJ,mod_flux,ls=plotting_dict[spec][1],color=plotting_dict[spec][0],label=spec)
ax[0].plot(waveGJ,fluxGJ,ls='-',color='k',alpha=1)
ax[0].legend(loc='lower center',fontsize=fontsize_tick, ncols=4)

# Making the plot pretty
ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].set_xlim(0.1,1.0)
xt = [0.1,0.2,0.3,0.4,0.5,0.75,1.00]
ax[0].set_xticks(xt)
ax[0].set_xticklabels([''.format(f) for f in xt])

from matplotlib.ticker import NullFormatter as nf
ax[0].tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax[0].xaxis.set_minor_formatter(nf())
ax[0].set_xlabel('',fontsize=fontsize_axis)
ax[0].set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]',fontsize=fontsize_axis)
plt.rc('grid',color='grey',alpha=0.5,ls=':')
ax[0].grid()

hands = []
for spec in tqdm(list(absorption_dict.keys()),
                desc='Computing absorption...'):
    if spec in ('C_III','O_II','O_III','Na_II','K_II'):
        continue
    wave, sigma, h = bin_cross_dict_H[spec]
    print(wave[-1]-wave[-2])
    print(wave[0]-wave[1])
    ok = (wave>0.1) & (wave<1.0)
    n_dens = ndens_dict[spec]
    f = n_dens * sigma * 3 * h
    f[f<0] = 0
    f[f>1] = 1
    print('column density of {}: {:.2E} cm-2'.format(spec,n_dens * 3 * h))
    print(wave)
    mindex = np.argmin(f)
    print(min(f),wave[mindex])
    absorptions[spec] = 1-f

    hand, = ax[1].plot(wave,100*(1-f),label=spec,
                       ls=plotting_dict[spec][1],
                       color=plotting_dict[spec][0]) # plot the absorption of each species by wavelength
    hands.append(hand)

legend1 = ax[1].legend(handles=hands,ncols=4,loc='lower center',fontsize=fontsize_tick)
ax[1].add_artist(legend1)

ax[1].set_xlabel('Wavelength [nm]',fontsize=fontsize_axis)
ax[1].set_ylabel('Observed/Expected Flux [%]',fontsize=fontsize_axis)
ax[1].set_yticks([100,95,90,85])
ax[1].set_xscale('log')
ax[1].set_xlim(0.10, 1.00)
ax[1].set_ylim(80,105)
ax[1].set_xticks([0.1,0.25,0.5,0.75,1.0])
ax[1].set_xticklabels(labels=['100','250','500','750','1000'])
ax[1].tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax[1].xaxis.set_minor_formatter(nf())
plt.rc('grid',color='grey',alpha=0.5,ls=':')
ax[1].grid(zorder=1)
plt.savefig('gj367_plot.png',dpi=300,bbox_inches='tight')
plt.show()
plt.close()

And you're done. :)

BONUS: Everything below is for generating the appendix figures.

In [343]:
### MORE EQUATIONS ###
def instel(Teff, aoR):
    return ((Teff/5772)**4)*((aoR/215.032)**-2)

def lumin(Teff, Rs):
    return (Rs**2)*((Teff/5772)**4)

def shore(v):
    return 1e-3*(v**4)

def xuv(I,L):
    return I*(L**-0.6)

In [344]:
### PLOTTING FUNC ###
def plot_thermal_shoreline(names,vescs,instels,rps,tides,Tstar):
    # Establish the template and draw the escape lines.
    x = np.linspace(1,100,1000)
    y = np.linspace(1,10000,1000)
    xs = [6,8,10,15,20,25]
    ys = [0.1,1.0,10.0,100.0,1000.0,10000.0]

    l,h = (20,10)
    fig, ax = plt.subplots(figsize=(l,h))
    theta = 0.335*np.arctan(l/h)*180/np.pi

    shoreline = shore(x)

    ax.plot(x,shoreline,color='red',lw=3,zorder=1)
    ax.text(6.9,2.5,s='VOLATILE-DEPLETED',rotation=theta,
            color='red',weight='bold',fontsize=fontsize_tick)

    # Now start putting in the planets.
    F = 3000
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    re_size = 0.75*F*np.log10(np.square(np.array(1))+1)
    im = ax.scatter(vescs,instels,s=sizes,c=Tstar,cmap=spectral_cmap,edgecolor='blue',lw=3,vmin=2500,vmax=6000)

    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower right',borderpad=1.0,labelspacing=3.0,fontsize=fontsize_tick)

    # Label them.
    for k in range(len(names)):
        if names[k] in best:
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='blue',marker='x',lw=1)
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='None',edgecolor='blue',lw=3)
    
        
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,1e10,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='upper center',ncols=3,columnspacing=0.5,handlelength=3,borderpad=2.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticks(xs)
    ax.set_xticklabels([str(i) for i in xs])
    ax.set_xlabel('Escape Velocity [km/s]',fontsize=fontsize_axis)    
    ax.set_xlim(6,25)
    ax.set_yticks(ys)
    ax.set_yticklabels([str(i) for i in ys])
    ax.set_ylabel('Instellation [w.r.t. Earth]',fontsize=fontsize_axis)
    for yi in ys:
        ax.axhline(y=yi,color='whitesmoke',zorder=0)
    for xi in xs:
        ax.axvline(x=xi,color='whitesmoke',zorder=0)
    ax.tick_params(axis='both',which='both',direction='in',labelsize=fontsize_tick)
    ax.set_ylim(1,10000)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)

    plt.savefig('thermal_shore.png',dpi=300,bbox_inches='tight')
    plt.close()

In [345]:
### PLOTTING FUNC ###
def plot_xuv_shoreline(names,vescs,instels,rps,tides,Tstar,rstar):
    # Establish the template and draw the escape lines.
    x = np.linspace(1,100,1000)
    y = np.linspace(1,10000,1000)
    xs = [6,8,10,15,20,25]
    ys = [0.1,1.0,10.0,100.0,1000.0,10000.0]

    l,h = (20,10)
    fig, ax = plt.subplots(figsize=(l,h))
    theta = 0.335*np.arctan(l/h)*180/np.pi

    shoreline = shore(x)

    ax.plot(x,shoreline,color='red',lw=3,zorder=1)
    ax.text(6.9,2.5,s='VOLATILE-DEPLETED',rotation=theta,
            color='red',weight='bold',fontsize=fontsize_tick)

    # Now start putting in the planets.
    F = 3000
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    re_size = 0.75*F*np.log10(np.square(np.array(1))+1)
    # Replace instels with XUV
    lumins = lumin(Tstar,rstar)
    instels = xuv(instels,lumins)
    im = ax.scatter(vescs,instels,s=sizes,c=Tstar,cmap=spectral_cmap,edgecolor='blue',lw=3,vmin=2500,vmax=6000)

    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower right',borderpad=1.0,labelspacing=3.0,fontsize=fontsize_tick)

    # Label them.
    for k in range(len(names)):
        if names[k] in best:
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='blue',marker='x',lw=1)
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='None',edgecolor='blue',lw=3)
    
        
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,1e10,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower center',ncols=3,columnspacing=0.5,handlelength=3,borderpad=2.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticks(xs)
    ax.set_xticklabels([str(i) for i in xs])
    ax.set_xlabel('Escape Velocity [km/s]',fontsize=fontsize_axis)    
    ax.set_xlim(6,25)
    ax.set_yticks(ys)
    ax.set_yticklabels([str(i) for i in ys])
    ax.set_ylabel('Cumulative XUV Flux [w.r.t Earth]',fontsize=fontsize_axis)
    for yi in ys:
        ax.axhline(y=yi,color='whitesmoke',zorder=0)
    for xi in xs:
        ax.axvline(x=xi,color='whitesmoke',zorder=0)
    ax.tick_params(axis='both',which='both',direction='in',labelsize=fontsize_tick)
    ax.set_ylim(1,10000)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)

    plt.savefig('xuv_shore.png',dpi=300,bbox_inches='tight')
    plt.close()

In [346]:
new_names,vescs,instels,new_rps,check_tides,Tstar,rstar,eccs,aors,alfvens,periods = [],[],[],[],[],[],[],[],[],[],[]
for pl in list(processed_output.keys()):
    planet = processed_output[pl]
    try:
        if pl == "Io":
            continue
        alfven, tau, tides, aor, teq, V, rp, mp, Teff, ecc_err = [planet['alfven'][0],
                                                                  planet['tau'][0],
                                                                  planet['tides'],
                                                                  planet['aor'],
                                                                  get_teq(planet['aor'],planet['Teff']),
                                                                  planet['V'],
                                                                  planet['rp'],
                                                                  planet['mp'],
                                                                  planet['Teff'],
                                                                  planet['ecc_err']]
        rs = planet['rstar']
        ecc = planet['ecc']
        period = planet['P']
        v = vesc(m=mp*masse,r=rp*rade)/(100*1000)
        i = instel(Teff,aor)
        if alfven < aor:
            print(1/0)
        if tides != 0:
            new_names.append(pl)
        else:
            print(1/0)
        vescs.append(v)
        instels.append(i)
        check_tides.append(tides)
        new_rps.append(rp)
        Tstar.append(Teff)
        rstar.append(rs)
        eccs.append(ecc)
        aors.append(aor)
        alfvens.append(alfven)
        periods.append(period*day)
    except:
        pass

plot_thermal_shoreline(new_names,np.array(vescs),np.array(instels),np.array(new_rps),np.array(check_tides),np.array(Tstar))
plot_xuv_shoreline(new_names,np.array(vescs),np.array(instels),np.array(new_rps),np.array(check_tides),np.array(Tstar),np.array(rstar))

Okay now you are definitely done. :)